In [17]:
to be tested

from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader

# 1. Load the Pretrained Model and Tokenizer
model_name = "roberta-base-openai-detector"
model = RobertaForSequenceClassification.from_pretrained(model_name)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# 2. Prepare Your Dataset
class MyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        encoding['labels'] = torch.tensor([label])
        return encoding

texts = ["This is an example text.", "Another example."]
labels = [0, 1]  # Use your actual labels here

# 3. Tokenize Your Dataset
dataset = MyDataset(texts, labels, tokenizer)

# 4. Train the Model
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset,               # training dataset
)

trainer.train()

# 5. Evaluate and Use the Model
# Now the model is fine-tuned, you can evaluate it on a validation set, or use it for predictions
print(trainer.predict(dataset))


SyntaxError: invalid syntax (846495780.py, line 1)

In [4]:
'''dataset'''


# Import all the needed libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import functools
import wandb
import random
import os

from datasets import Dataset, DatasetDict, load_metric

from sklearn.metrics import classification_report, confusion_matrix

from keras.callbacks import EarlyStopping

from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
 TrainingArguments, Trainer, pipeline, EarlyStoppingCallback, \
 EncoderDecoderModel, RobertaTokenizerFast

'''Loading data'''
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split


df = pd.read_json(os.getcwd()+'/SubtaskA/subtaskA_train_monolingual.jsonl', lines=True)
# Just interested so far in text and label
df = df[['text', 'label']]

'So testing doesnt takes too much time processing, 10k seems ok'
df=df.sample(20000)

print('\nExample of dataframe (text|label)\n')
print(df.sample(5))
print(f'\nSize{df.shape}')
print(f'\nValue count \n')
print(df['label'].value_counts())

# Splitting the DataFrame into train, dev, and test sets
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)  # 60% training, 40% temporary
dev_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)  # 20% development, 20% test



Example of dataframe (text|label)

                                                     text  label
114918    We derive masses and radii for both componen...      0
116652    Photon-induced reactions play a key role in ...      0
119223  This work finds a connection between Bourgain'...      0
31412   Well, um, eggs are really yummy, but some peop...      1
106375    Properties of planetary atmospheres, ionosph...      0

Size(20000, 2)

Value count 

label
0    10642
1     9358
Name: count, dtype: int64


In [24]:
import tensorflow as tf

train_text=train_df['text'].tolist()
train_label=train_df['label'].tolist()

val_text=dev_df['text'].tolist()
val_label=dev_df['label'].tolist()

test_text=test_df['text'].tolist()
test_label=test_df['label'].tolist()

MODEL_NAME = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME )

train_encodings = tokenizer(train_text, truncation=True, padding=True)
val_encodings = tokenizer(val_text, truncation=True, padding=True)
test_encodings = tokenizer(test_text, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_label
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_label
))

train_dataset


<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [21]:
'''model'''

from transformers import AutoModel

model=AutoModel.from_pretrained(MODEL_NAME)



In [26]:
'''training'''
from transformers import TFTrainingArguments, TFTrainer

training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,              
    logging_dir='./logs',            
    logging_steps=10,
)


trainer = TFTrainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset             
)


c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\trainer_tf.py:118: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  warnings.warn(


In [27]:
trainer.train()

ValueError: `Checkpoint` was expecting model to be a trackable object (an object derived from `Trackable`), got DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=